In [1]:
#! pip install google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 KB 934.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 KB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 KB 4.5 MB/s eta 0:00:0000:01


# Required Libraries and Setup config

In [13]:
import pandas as pd
from datetime import datetime
from google.cloud import storage
import subprocess
import os
import getpass


timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
environment = 'prod'

# bucket information 
bucket_name = 'gmb-database-backup'
backup_destination_path = f'{environment}/backups/{timestamp}/'

username = 'gmb'
public_ip_address = '34.133.118.52'
db_name = 'gmb'
port = '5432'
password = getpass.getpass('Enter password:')


# postgresql configuration
db_name = db_name
username = username
host = public_ip_address
port = port


Enter password:········


### Create a Bucket

In [15]:
def create_bucket():
    storage_client = storage.Client()
    
    if not storage_client.lookup_bucket(bucket_name):
        bucket = storage_client.create_bucket(bucket_name)
        print(f"Bucket {bucket_name} created.")
    else:
        print(f"Bucket {bucket_name} already exists.")

### Create the Database Backup

In [16]:
def create_backup():
    backup_fname = '{environment}_backup_{timestamp}.sql'
    os.environ['PGPASSWORD'] = password
    command = f'pg_dump -U {username} -h {host} -p {port} {db_name} > {backup_fname}'
    
    result = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    
    if result.returncode != 0:
        print("Error creating backup:", result.stderr.decode())
        return None
    
    return backup_file_name

### Upload the Backup to GCS

In [17]:
def upload_to_gcs(file_path):
    
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(os.path.join(backup_destination_path, os.path.basename(file_path)))
    
    blob.upload_from_filename(file_path)
    
    print(f"File {file_path} uploaded to {bucket_name}/{backup_destination_path}.")

In [18]:
backup_file = create_backup()
if backup_file:
    upload_to_gcs(backup_file)

Error creating backup: pg_dump: server version: 15.2; pg_dump version: 11.5
pg_dump: aborting because of server version mismatch

